In [1]:
import tensorflow as tf
import numpy as np
import nltk
import gensim as gs

In [2]:
def read_file():#read whole document
    file_obj = open('ihavedream','r')
    my_text = file_obj.read(-1)
    my_text = my_text.replace(',',' ,').replace('.',' . ').replace('\n\n','').replace('"',' " ').replace('?',' ? ').replace('!',' ! ')
    file_obj.close()
    return my_text

def sent2word(sentences):#convert sentences list to word list but persist order
    sent_words = []
    for sentence in sentences:
        tokenized_text = nltk.word_tokenize(sentence)
        sent_words.append(tokenized_text)
    return sent_words

def word2vector(words,gsmodel):
    vec = []
    for word in words:
        for wor in word:
            temp = gsmodel.wv[wor]
            vec.append(temp)
    return vec
def vector2word(vec,gsmodel):
    return model.wv.most_similar(positive=vec,topn=1)[0][0]

In [3]:
document = read_file()#the whole documents
sentences = nltk.sent_tokenize(document)
words = sent2word(sentences)
model = gs.models.Word2Vec(words, min_count=0,size=100)
result = word2vector(words,model)
X = result[0:1843-1]
Y = result[1:]

In [4]:
def fc_layer(w,b,inputs):
    return tf.add(tf.matmul(inputs,w),b)

In [5]:
tfX = tf.placeholder(shape=[1,100],dtype=tf.float32,name='tfX')
tfY = tf.placeholder(shape=[1,100],dtype=tf.float32,name='tfY')

tfW = tf.Variable(tf.constant(0,shape=[100,100],dtype=tf.float32),name='tfW')
tfB = tf.Variable(tf.constant(0,shape=[1,100],dtype=tf.float32),name='tfB')

In [6]:
rnncell = tf.nn.rnn_cell.BasicRNNCell(num_units=100,activation=tf.nn.tanh)
h = rnncell.zero_state(1,dtype=tf.float32)
output1, h = rnncell.call(tfX,h)

In [7]:
output2, h2 = rnncell.call(tfX,h)
res = fc_layer(tfW,tfB,h2)
loss = tf.losses.mean_squared_error(labels=tfY,predictions=res)

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
init_h = sess.run(h,feed_dict={tfX:X[0].reshape(1,100)})

In [10]:
epoch = 50
for _ in range(epoch):
    result_loss = 0
    for i,x in enumerate(X):
        if i >= 1:
            dictionary = {tfX:X[i].reshape(1,100),tfY:Y[i].reshape(1,100),h:init_h}
            init_h = sess.run(h2,feed_dict=dictionary)
            sess.run(optimizer,feed_dict=dictionary)
            W = sess.run(tfW)
            B = sess.run(tfB)
            H = init_h
            result_loss = sess.run(loss,feed_dict=dictionary)
            result_loss += result_loss
    print(result_loss)

1.42027e-05
1.38597e-05
1.26431e-05
1.20042e-05
1.18306e-05
1.18579e-05
1.19835e-05
1.21454e-05
1.20893e-05
1.18902e-05
1.14697e-05
1.10419e-05
1.07333e-05
1.05577e-05
1.03762e-05
1.00369e-05
9.7131e-06
9.6874e-06
9.73182e-06
9.72323e-06
9.65283e-06
9.60429e-06
9.83907e-06
1.01739e-05
1.01545e-05
9.92364e-06
9.71716e-06
9.64616e-06
9.74883e-06
9.6878e-06
9.53443e-06
9.45787e-06
9.5063e-06
9.55342e-06
9.57016e-06
9.51714e-06
9.4594e-06
9.42836e-06
9.41392e-06
9.48922e-06
9.60539e-06
9.58835e-06
9.38794e-06
9.17315e-06
9.07264e-06
9.07983e-06
9.12416e-06
9.17171e-06
9.21392e-06
9.34986e-06


In [11]:
sumary = tf.summary.FileWriter('./test',sess.graph)
sumary.close()
sess.close()

In [12]:
tf.reset_default_graph()

In [13]:
#prediction
pred_W = tf.Variable(W)
pred_B = tf.Variable(B)
pred_H = tf.Variable(H)
pred_X = tf.Variable(X[0].reshape(1,100))

pred_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=100)
pred_output,_ = pred_cell.call(pred_X,pred_H)
pred_X_old = fc_layer(pred_W,pred_B,pred_output)

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [15]:
for i in range(100):
    res_X = sess.run(pred_X_old)
    pred_X.load(res_X,session=sess)
    pred_wordis = vector2word(res_X,model)
    print(pred_wordis)

we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we
we


In [53]:
sess.close()